# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Defaulting to user installation because normal site-packages is not writeable


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位很棒的生物家。你擅长回答数生物问题。
你擅长以清晰透彻的方式回答关于生物学的问题。
当你不确定某个问题的答案时，你会诚实地说出来。

这是一个问题：
{input}"""

computer_template = """你是一位经验丰富的计算机科学专家。
你擅长以逻辑严谨的方式回答关于计算机科学的问题。
当你遇到一个未知的问题时，你会勇于承认并寻找答案。

这是一个问题：
{input}"""

chinese_template = """你是一位深受学生喜爱的中文老师。
你擅长以生动有趣的方式回答关于中文和文学的问题。
如果遇到你不了解的问题，你会坦率地表示并尽力寻找解答。

这是一个问题：
{input}"""

In [3]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
        {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
        {
        "name": "汉语文",
        "description": "适用于回答汉语文问题",
        "prompt_template": chinese_template,
    },

]

In [4]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [5]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [6]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [7]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [8]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [9]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语文: 适用于回答汉语文问题']


In [10]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语文: 适用于回答汉语文问题


In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语文: 适用于回答汉语文问题

<< INPUT >>
{input}

<< OU

In [13]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [14]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指一个完美吸收所有电磁波的物体，在不同温度下会发射出不同波长的电磁波，这种现象被称为黑体辐射。它是研究热辐射的基础，也是理解太阳和其他天体的热辐射的重要概念。'}


In [15]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': 'What is the first prime number greater than 40?'}
> Finished chain.
{'input': 'What is the first prime number greater than 40?', 'text': '\n\n首先，我们需要知道什么是质数。质数是指只能被1和自身整除的自然数。\n因此，我们需要从41开始依次检查每个数字是否为质数。\n\n41是质数，因此它就是大于40的第一个质数。如果需要继续寻找更大的质数，\n我们可以按照同样的步骤继续检查比41更大的数字，直到找到满足条件的质数。'}


In [16]:
router_chain.verbose = True

In [17]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是宇宙中非常密集的物质聚集体，它具有如此强大的引力，即使是光也无法逃脱其吸引力。这使得黑洞看起来像是一个看不见的“洞”，因为它吸收了周围的所有物质和光线。它的存在是由于恒星的死亡，当恒星耗尽了燃料后，它会坍缩成一个极其密集的物体，形成黑洞。黑洞是宇宙中最神秘的现象之一，它们具有极高的密度和强大的引力，但是我们仍在研究它们的奥秘。'}


In [18]:
print(chain.invoke("人是由什么组成的？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '人是由什么组成的？'}
> Finished chain.
{'input': '人是由什么组成的？', 'text': '\n\n人是由细胞组成的。细胞是生物体的基本结构单位，所有的生物体都是由一个或多个细胞组成的。细胞可以分为原生质和细胞器，其中原生质包括细胞膜、细胞质和细胞核，而细胞器则包括线粒体、内质网等。细胞的组合和功能不同，可以形成不同类型的组织和器官，最终构成一个完整的生物体。'}


In [19]:
print(chain.invoke("世界上最好的编程语言是什么"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '世界上最好的编程语言是什么'}
> Finished chain.
{'input': '世界上最好的编程语言是什么', 'text': '？\n\n这是一个非常主观的问题，没有一个确定的答案。每个人对于最好的编程语言都有不同的看法，这取决于他们的背景、需求和个人偏好。一些人可能认为Java是最好的，因为它具有广泛的应用和强大的生态系统。其他人可能会说Python是最好的，因为它简单易学，具有丰富的库和框架。还有人可能会认为C++是最好的，因为它可以实现高性能和底层控制。因此，最好的编程语言是因人而异的，没有一个绝对的答案。重要的是根据自己的需求和兴趣选择适合自己的编程语言。'}


In [20]:
print(chain.invoke("床前明月光，下一句是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语文: {'input': '床前明月光，疑是地上霜'}
> Finished chain.
{'input': '床前明月光，疑是地上霜', 'text': '。\n举头望明月，低头思故乡。\n\n这是一个关于中国古诗词的问题。这首诗叫做《静夜思》，是唐代诗人李白的作品，描写了一个人在月光下思念故乡的情景。它也是中国人最熟悉的诗歌之一，经常被用来比喻思念家乡的情感。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains